# 'Kontaktverbot' vor und nach Covid-19 - eine Untersuchung mit Word2Vec

#### Bedeutungen des Lemmas 'Kontaktverbot'

Verbot des Kontaktes zu bestimmten Personen, Personengruppen oder zur Außenwelt <br>
a) als Maßnahme des Opferschutzes <br>
b) als Seuchenschutzmaßnahme <br>
c) zur Isolierung von Inhaftierten, Suchtkranken o. Ä. <br> 
(Quelle: https://www.dwds.de/wb/Kontaktverbot) <br> 
<br> 

#### Hypothese

Vor der Covid-19-Pandemie war die Lesart a) am verbreitetsten. Durch die Pandemie und die damit einhergehende Notwendigkeit von Seuchenschutzmaßnahmen gewinnt die Lesart b) an Popularität und verdrängt die anderen beiden Bedeutungen. Die Bedeutung des Lemmas wandelt sich. <br>
<br>
Anmerkung: Tatsache ist, dass die Pandemie aktuell für eine sehr ausgeprägte Verwendung der Lesart b) führt, die auch im mentalen Lexikon der Sprecher fest verankert ist, während diese Postion vorher Lesart a) innehatte. Ob tatsächlich ein Bedeutungswandel in all seinen Phasen stattfindet, bleibt abzuwarten.

## Importe und Datenvorbereitung

In [1]:
import codecs
import matplotlib.pyplot as plt
import nltk
import numpy as np
import os
import pandas as pd
import re
import scipy
import spacy
import seaborn as sns

from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from gensim.models.phrases import Phraser, Phrases
from joblib import Parallel, delayed  
from nltk.corpus import stopwords
from scipy import spatial
from sklearn.manifold import TSNE
from tabulate import tabulate

In [2]:
nlp = spacy.load('de_core_news_md')
stopwords = stopwords.words('german')
tokenizer = nltk.data.load('tokenizers/punkt/german.pickle')

#### Korpus laden und teilen

In [3]:
# die Datei enthält alle Treffer des Lemmas 'kontaktverbot' im ZDL-Regionalkorpus des DWDS 
# mit jeweils einem Satz Kontext davor und danach

df = pd.read_csv('../data/kontaktverbot_1993-2021.csv', sep=',', index_col='No.', encoding='utf-8')
df.head()

,Corpus,Date,Genre,Bibl,ContextBefore,Hit,ContextAfter
No.,,,,,,,
1,saar_regional,1996-08-31,Zeitung,"Saarbrücker Zeitung, 31.08.1996",Der Vater ist inzwischen aus seinem eigenen Ha...,"Die Mutter hat durchgesetzt, daß der Vater Kon...","Sie weiß, daß er oft im Ort unterwegs ist. ent..."
2,saar_regional,1996-09-11,Zeitung,"Saarbrücker Zeitung, 11.09.1996","Der Täter bleibt in der Wohnung, in seinem nor...",In den USA müsse der Täter ausziehen und erhal...,Nach der engagierten Diskussion trat Bettina K...
3,rztg_regional,1997-02-25,Zeitung,"Rhein-Zeitung, 25.02.1997","[...]n, nachdem sie eines der Opfer gedrängt h...",Der dritte Beschuldigte habe ein absolutes Kon...,Zu den Beschuldigten gehört nach US-Quellen au...
4,fr_regional,1997-03-03,Zeitung,"Frankfurter Rundschau, 03.03.1997","[...]gen zurückzunehmen. R., dem unter anderem...",Ihm sei aber strenges Kontaktverbot auferlegt ...,NaN
5,fr_regional,1997-12-13,Zeitung,"Frankfurter Rundschau, 13.12.1997","[...]ungsauflagen der Justiz, ohne daß das für...",Weil der Polizei aber Auflagen nicht mitgeteil...,"Herrmann: ""Selbst erneut begangene Straftaten ..."


In [4]:
# NaN durch Whitespace ersetzen

df = df.fillna(' ')

In [5]:
# bevor die Spalten verbunden werden: Whitespace einfügen, um ein Aneinanderkleben der Wörter zu verhindern

df['ContextBefore'] = df['ContextBefore'].astype(str) + ' '
df['ContextAfter'] = ' ' + df['ContextAfter'].astype(str) 

In [6]:
# Text zu einer Spalte verbinden

columns = ['ContextBefore', 'Hit', 'ContextAfter']

df['Text'] = df[columns].astype(str).sum(axis=1)
df.head()

,Corpus,Date,Genre,Bibl,ContextBefore,Hit,ContextAfter,Text
No.,,,,,,,,
1,saar_regional,1996-08-31,Zeitung,"Saarbrücker Zeitung, 31.08.1996",Der Vater ist inzwischen aus seinem eigenen Ha...,"Die Mutter hat durchgesetzt, daß der Vater Kon...","Sie weiß, daß er oft im Ort unterwegs ist. en...",Der Vater ist inzwischen aus seinem eigenen Ha...
2,saar_regional,1996-09-11,Zeitung,"Saarbrücker Zeitung, 11.09.1996","Der Täter bleibt in der Wohnung, in seinem nor...",In den USA müsse der Täter ausziehen und erhal...,Nach der engagierten Diskussion trat Bettina ...,"Der Täter bleibt in der Wohnung, in seinem nor..."
3,rztg_regional,1997-02-25,Zeitung,"Rhein-Zeitung, 25.02.1997","[...]n, nachdem sie eines der Opfer gedrängt h...",Der dritte Beschuldigte habe ein absolutes Kon...,Zu den Beschuldigten gehört nach US-Quellen a...,"[...]n, nachdem sie eines der Opfer gedrängt h..."
4,fr_regional,1997-03-03,Zeitung,"Frankfurter Rundschau, 03.03.1997","[...]gen zurückzunehmen. R., dem unter anderem...",Ihm sei aber strenges Kontaktverbot auferlegt ...,,"[...]gen zurückzunehmen. R., dem unter anderem..."
5,fr_regional,1997-12-13,Zeitung,"Frankfurter Rundschau, 13.12.1997","[...]ungsauflagen der Justiz, ohne daß das für...",Weil der Polizei aber Auflagen nicht mitgeteil...,"Herrmann: ""Selbst erneut begangene Straftaten...","[...]ungsauflagen der Justiz, ohne daß das für..."


In [11]:
# Korpus in 2 Teilkorpora splitten: vor und nach Covid-19 (dem Ausbruch in Deutschland)
# hier wurde die Grenze zwischen Januar und Februar 2020 gezogen (es wird sich noch zeigen, inwiefern sich das bewährt)

df_before_corona = df.iloc[:1421,:]
df_after_corona = df.iloc[1422:,:]

In [12]:
df_after_corona.head()

,Corpus,Date,Genre,Bibl,ContextBefore,Hit,ContextAfter,Text
No.,,,,,,,,
1423,maz_regional,2020-02-01,Zeitung,"Allgemeine Zeitung, 01.02.2020",[...]Roten Karte gegen einen Trainer.,"Sandro Schwarz, ehemaliger Coach des FSV Mainz...",,[...]Roten Karte gegen einen Trainer. Sandro S...
1424,frt_regional,2020-02-03,Zeitung,"Fränkischer Tag, 03.02.2020",Er konnte jedoch kurz darauf kontaktiert werden.,Neben einer Strafanzeige wegen Bedrohung und S...,,Er konnte jedoch kurz darauf kontaktiert werde...
1425,kn_regional,2020-02-14,Zeitung,"Kieler Nachrichten, 14.02.2020",[...]e indem Angreifer des Wohnortes verwiesen...,Bieler zufolge war der 24-Jährige mit einem Ko...,Dagegen habe er verstoßen.,[...]e indem Angreifer des Wohnortes verwiesen...
1426,kn_regional,2020-02-15,Zeitung,"Kieler Nachrichten, 15.02.2020",[...]r Vater des neben dem Opfer gefundenen Sä...,Zwar hatte das Amtsgericht Rendsburg gegen ihn...,Doch beide checkten zusammen in das Hotel in ...,[...]r Vater des neben dem Opfer gefundenen Sä...
1427,ta_regional,2020-02-20,Zeitung,"Thüringer Allgemeine, 20.02.2020",Zu Details folgten keine Angaben.,"Denkbar sind aber Verlegungen, Kontaktverbote ...",Die Massenschlägerei gebe aus Sicht der Behör...,Zu Details folgten keine Angaben. Denkbar sind...


Anmerkung: <br>
- Duplikate! 
<br> --> rauswerfen?

#### Hilfsfunktionen zur Vorbereitung der Texte

In [13]:
def lemmatize_text_column(df, column):
    """
    transforms the Dataframe-column in a lemmatized string
    """
    text = ''
    for i in df[column]:
        doc = nlp(i)
        lemmas = ' '.join([x.lemma_ for x in doc])
        text = text + lemmas
    return text


def sentence_to_wordlist(raw:str):
    """
    cleans and tokenizes the sentences
    """
    text = re.sub('[^A-Za-z_äÄöÖüÜß]',' ', raw).split()
    filtered_text = [word for word in text if word not in stopwords]
    return filtered_text


def prepare_text(raw_text):
    """
    returns a list of tokenized sentences
    """
    raw_sentences = tokenizer.tokenize(str(raw_text).lower())    
    tokenized_sentences = Parallel(n_jobs=-1)(delayed(sentence_to_wordlist)(raw_sentence) for raw_sentence in raw_sentences)
    phrases = Phrases(tokenized_sentences)
    bigram = Phraser(phrases)
    sentences = list(bigram[tokenized_sentences])
    return sentences

#### Vorbereitung des ersten Texts (vor Covid-19)

In [14]:
text = lemmatize_text_column(df_before_corona, 'Text')
sentences = prepare_text(text)

# sentences ist eine Liste von tokenisierten Sätzen, zum Beispiel:
print(sentences[15])

['urteil', 'streng_auflage', 'kontaktverbot', 'therapiekontrolle', 'schwester', 'bleiben', 'aussage', 'gericht', 'ersparen']


#### Vorbereitung des zweiten Texts (nach Covid-19)

In [16]:
text2 = lemmatize_text_column(df_after_corona, 'Text')
sentences2 = prepare_text(text2)

print(sentences2[15])

['mai', 'sowie']


In [17]:
print(sentences2[100])

['sonntag', 'bundesweit', 'kontaktverbot', 'verschärfen', 'birgit', 'schellbach', 'jung', 'einschränkung', 'bundes', 'landesregierung', 'sonntag', 'verständigen', 'kontaktverbote', 'gelten', 'konsequenz', 'logisch']


## Training von Word2Vec auf den Text vor Covid-19

In [18]:
# Paramter setzen
workers = 4                      # Use these many worker threads to train the model (=faster training with multicore machines)
seed = 42                        # Seed for the random number generator

In [19]:
# Ordner anlegen zum Abspeichern von trainierten Modellen
if not os.path.exists('../trained_models'):
    os.makedirs('../trained_models')

In [20]:
# Training

w2v_bc = Word2Vec(sentences=sentences,                   
                 vector_size=300,                 # Dimensionality of the word vectors
                 window=10,                # The maximum distance between the current and predicted word within a sentence
                 min_count=3,              # (int, optional) – The model ignores all words with total frequency lower than this
                 workers=workers, 
                 min_alpha=0.0001,         # Learning rate will linearly drop to min_alpha as training progresses
                 sg=1,                     # Training algorithm: skip-gram if sg=1, otherwise CBOW
                 seed=seed)

## Training von Word2Vec auf den Text nach Covid-19

In [21]:
w2v_ac = Word2Vec(sentences=sentences2,                   
                 vector_size=300,                
                 window=10,              
                 min_count=3,             
                 workers=workers, 
                 min_alpha=0.0001,                                                    
                 sg=1,                     
                 seed=seed)

In [22]:
# trainierte Modelle speichern
w2v_bc.save(os.path.join('../trained_models', 'w2v_bc_kontaktverbot.model'))
w2v_ac.save(os.path.join('../trained_models', 'w2v_ac_kontaktverbot.model'))

## Exploration und Vergleich der Embeddings

In [23]:
# trainierte Modelle laden
w2v_bc = Word2Vec.load(os.path.join('../trained_models', 'w2v_bc_kontaktverbot.model'))
w2v_ac = Word2Vec.load(os.path.join('../trained_models', 'w2v_ac_kontaktverbot.model'))

In [24]:
# ähnliche Wörter zu 'kontaktverbot' vor Covid-19
w2v_bc.wv.most_similar(positive=['kontaktverbot'], topn=25)

[('straße', 0.9989418983459473),
 ('zeit', 0.9989309310913086),
 ('ehemann', 0.9989079833030701),
 ('jährigen', 0.9989057779312134),
 ('strafanzeige', 0.9988993406295776),
 ('gerichtlich', 0.9988943934440613),
 ('zunächst', 0.9988943338394165),
 ('anzeige_wegen', 0.9988898634910583),
 ('mann', 0.9988876581192017),
 ('kontaktverbot_aussprechen', 0.9988860487937927),
 ('gegenüber', 0.9988839030265808),
 ('lassen', 0.9988782405853271),
 ('mal', 0.9988769292831421),
 ('gewaltschutzgesetz', 0.9988694787025452),
 ('kommen', 0.9988656044006348),
 ('obwohl', 0.99886554479599),
 ('polizei', 0.9988542199134827),
 ('jed', 0.998848021030426),
 ('angabe', 0.9988469481468201),
 ('vorläufig', 0.998845100402832),
 ('bad', 0.9988449215888977),
 ('beim', 0.9988433122634888),
 ('zurück', 0.9988423585891724),
 ('verlassen', 0.9988422989845276),
 ('zuvor', 0.9988399147987366)]

In [25]:
# ähnliche Wörter zu 'kontaktverbot' nach Covid-19
w2v_ac.wv.most_similar(positive=['kontaktverbot'], topn=25)

[('all', 0.9947583079338074),
 ('halten', 0.9945807456970215),
 ('kontaktverbote', 0.994234561920166),
 ('gelten', 0.993869960308075),
 ('regel', 0.9937571883201599),
 ('einhalten', 0.9936049580574036),
 ('streng', 0.993476927280426),
 ('strikt', 0.9934644103050232),
 ('mensch', 0.9933945536613464),
 ('maßnahme', 0.9933112263679504),
 ('märz', 0.9931747317314148),
 ('groß', 0.992999792098999),
 ('weit', 0.9929946064949036),
 ('verhängen', 0.9929713010787964),
 ('mehr', 0.992878258228302),
 ('müssen', 0.992827296257019),
 ('gruppe', 0.992824912071228),
 ('erlassen', 0.9925863742828369),
 ('einschränkung', 0.9924989342689514),
 ('öffentlich_raum', 0.9924449324607849),
 ('geltend', 0.9924158453941345),
 ('kontrollieren', 0.9924034476280212),
 ('einhaltung', 0.9923020005226135),
 ('bleiben', 0.9922780990600586),
 ('zudem', 0.9922617673873901)]

In [26]:
w2v_ac.wv.most_similar(positive=['virus'], topn=10)

[('auswirkung', 0.9992698431015015),
 ('nie', 0.9992685914039612),
 ('fehlend', 0.9992489218711853),
 ('v', 0.9992386698722839),
 ('osterferien', 0.9992345571517944),
 ('wirken', 0.9992325305938721),
 ('mittel', 0.9992254972457886),
 ('verhindern', 0.9992189407348633),
 ('lockerung', 0.9992176294326782),
 ('ziel', 0.9992175698280334)]

### Weiteres Vorgehen

- Optimierung der Hyperparameter von Word2Vec?
- Ausrichtung der beiden Embedding-Modelle, um Vergleichbarkeit zu schaffen (d.h. die gleiche Art von semantischen Informationen müssen in übereinstimmenden Dimensionen codiert sein)
- Vergleich der Vektoren (Cosinus-Similarität bzw. -Abstand)
- Überprüfung anhand anderer Vektoren und durch Samplen der Korpora

## Ausrichtung der beiden Embedding-Modelle

In [27]:
# Quelle: https://gist.github.com/zhicongchen/9e23d5c3f1e5b1293b16133485cd17d8

def smart_procrustes_align_gensim(base_embed, other_embed, words=None):
    """
    Original script: https://gist.github.com/quadrismegistus/09a93e219a6ffc4f216fb85235535faf
    Updatet script: https://gist.github.com/zhicongchen/9e23d5c3f1e5b1293b16133485cd17d8
    Procrustes align two gensim models (to allow for comparison between same word across models).
    Code ported from HistWords <https://github.com/williamleif/histwords> by William Hamilton <wleif@stanford.edu>.
        
    First, intersect the vocabularies (see `intersection_align_gensim` documentation).
    Then do the alignment on the other_embed model.
    Replace the other_embed model's syn0 and syn0norm numpy matrices with the aligned version.
    Return other_embed.

    If `words` is set, intersect the two models' vocabulary with the vocabulary in words (see `intersection_align_gensim` documentation).
    """

    # patch by Richard So [https://twitter.com/richardjeanso) (thanks!) to update this code for new version of gensim
    # base_embed.init_sims(replace=True)
    # other_embed.init_sims(replace=True)

    # make sure vocabulary and indices are aligned
    in_base_embed, in_other_embed = intersection_align_gensim(base_embed, other_embed, words=words)
    
    # re-filling the normed vectors
    in_base_embed.wv.fill_norms(force=True)
    in_other_embed.wv.fill_norms(force=True)

    # get the (normalized) embedding matrices
    base_vecs = in_base_embed.wv.get_normed_vectors()
    other_vecs = in_other_embed.wv.get_normed_vectors()

    # just a matrix dot product with numpy
    m = other_vecs.T.dot(base_vecs) 
    # SVD method from numpy
    u, _, v = np.linalg.svd(m)
    # another matrix operation
    ortho = u.dot(v) 
    # Replace original array with modified one, i.e. multiplying the embedding matrix by "ortho"
    other_embed.wv.vectors = (other_embed.wv.vectors).dot(ortho)    
    
    return other_embed

def intersection_align_gensim(m1, m2, words=None):
    """
    Intersect two gensim models, m1 and m2.
    Only the shared vocabulary between them is kept.
    If 'words' is set (as list or set), then the vocabulary is intersected with this list as well.
    Indices are re-organized from 0..N in order of descending frequency (=sum of counts from both m1 and m2).
    These indices correspond to the new syn0 and syn0norm objects in both gensim models:
        -- so that Row 0 of m1.syn0 will be for the same word as Row 0 of m2.syn0
        -- you can find the index of any word on the .index2word list: model.index2word.index(word) => 2
    The .vocab dictionary is also updated for each model, preserving the count but updating the index.
    """

    # Get the vocab for each model
    vocab_m1 = set(m1.wv.index_to_key)
    vocab_m2 = set(m2.wv.index_to_key)

    # Find the common vocabulary
    common_vocab = vocab_m1 & vocab_m2
    if words: common_vocab &= set(words)

    # If no alignment necessary because vocab is identical...
    if not vocab_m1 - common_vocab and not vocab_m2 - common_vocab:
        return (m1,m2)

    # Otherwise sort by frequency (summed for both)
    common_vocab = list(common_vocab)
    common_vocab.sort(key=lambda w: m1.wv.get_vecattr(w, "count") + m2.wv.get_vecattr(w, "count"), reverse=True)
    # print(len(common_vocab))

    # Then for each model...
    for m in [m1, m2]:
        # Replace old syn0norm array with new one (with common vocab)
        indices = [m.wv.key_to_index[w] for w in common_vocab]
        old_arr = m.wv.vectors
        new_arr = np.array([old_arr[index] for index in indices])
        m.wv.vectors = new_arr

        # Replace old vocab dictionary with new one (with common vocab)
        # and old index2word with new one
        new_key_to_index = {}
        new_index_to_key = []
        for new_index, key in enumerate(common_vocab):
            new_key_to_index[key] = new_index
            new_index_to_key.append(key)
        m.wv.key_to_index = new_key_to_index
        m.wv.index_to_key = new_index_to_key
        
        print(len(m.wv.key_to_index), len(m.wv.vectors))
        
    return (m1,m2)

In [28]:
# Ausrichtung des "After-Corona-Modells" an das "Before-Corona-Modell"

w2v_ac_al = smart_procrustes_align_gensim(w2v_bc, w2v_ac)

1043 1043
1043 1043


In [29]:
# speichern
w2v_ac_al.save(os.path.join('../trained_models', 'w2v_ac_al_kontaktverbot.model'))

In [30]:
# laden
w2v_ac_al = Word2Vec.load(os.path.join('../trained_models', 'w2v_ac_al_kontaktverbot.model'))

In [31]:
# ähnliche Wörter zu 'kontaktverbot' nach Covid-19
w2v_ac_al.wv.most_similar(positive=['kontaktverbot'], topn=25)

[('all', 0.9947583079338074),
 ('halten', 0.9945807456970215),
 ('kontaktverbote', 0.9942345023155212),
 ('gelten', 0.993869960308075),
 ('regel', 0.9937571287155151),
 ('einhalten', 0.9936049580574036),
 ('streng', 0.993476927280426),
 ('mensch', 0.9933945536613464),
 ('maßnahme', 0.9933112859725952),
 ('märz', 0.9931746125221252),
 ('groß', 0.992999792098999),
 ('weit', 0.9929947257041931),
 ('verhängen', 0.9929714202880859),
 ('mehr', 0.9928783178329468),
 ('müssen', 0.9928274154663086),
 ('erlassen', 0.9925863146781921),
 ('kontrollieren', 0.9924033880233765),
 ('einhaltung', 0.9923020005226135),
 ('bleiben', 0.9922780990600586),
 ('zudem', 0.9922617673873901),
 ('verstoßen', 0.9922390580177307),
 ('treffen', 0.9922292232513428),
 ('seit', 0.9921423196792603),
 ('trotzen', 0.9920843243598938),
 ('bereits', 0.9920445680618286)]

In [32]:
vector_kontaktverbot_bc = w2v_bc.wv['kontaktverbot']  
vector_kontaktverbot_ac_al = w2v_ac_al.wv['kontaktverbot'] 

cosine_kontaktverbot = 1 - spatial.distance.cosine(vector_kontaktverbot_bc, vector_kontaktverbot_ac_al)
cosine_kontaktverbot

0.9928577542304993

In [34]:
vector_deutschland_bc = w2v_bc.wv['deutschland']  
vector_deutschland_ac_al = w2v_ac_al.wv['deutschland'] 

cosine_deutschland = 1 - spatial.distance.cosine(vector_deutschland_bc, vector_deutschland_ac_al)
cosine_deutschland

0.9994009137153625

In [35]:
vector_nachricht_bc = w2v_bc.wv['nachricht']  
vector_nachricht_ac_al = w2v_ac_al.wv['nachricht'] 

cosine_nachricht = 1 - spatial.distance.cosine(vector_nachricht_bc, vector_nachricht_ac_al)
cosine_nachricht

0.9995245933532715

In [36]:
vector_schreiben_bc = w2v_bc.wv['schreiben']  
vector_schreiben_ac_al = w2v_ac_al.wv['schreiben'] 

cosine_schreiben = 1 - spatial.distance.cosine(vector_schreiben_bc, vector_schreiben_ac_al)
cosine_schreiben

0.9993204474449158

In [37]:
vector_sagen_bc = w2v_bc.wv['sagen']  
vector_sagen_ac_al = w2v_ac_al.wv['sagen'] 

cosine_sagen = 1 - spatial.distance.cosine(vector_sagen_bc, vector_sagen_ac_al)
cosine_sagen

0.9985068440437317

In [38]:
vector_mann_bc = w2v_bc.wv['mann']  
vector_mann_ac_al = w2v_ac_al.wv['mann'] 

cosine_mann = 1 - spatial.distance.cosine(vector_mann_bc, vector_mann_ac_al)
cosine_mann

0.9988973140716553

In [39]:
vector_frau_bc = w2v_bc.wv['frau']  
vector_frau_ac_al = w2v_ac_al.wv['frau'] 

cosine_frau = 1 - spatial.distance.cosine(vector_frau_bc, vector_frau_ac_al)
cosine_frau

0.9971783757209778

In [40]:
vector_mensch_bc = w2v_bc.wv['mensch']  
vector_mensch_ac_al = w2v_ac_al.wv['mensch'] 

cosine_mensch = 1 - spatial.distance.cosine(vector_mensch_bc, vector_mensch_ac_al)
cosine_mensch

0.9954843521118164

In [41]:
vector_montag_bc = w2v_bc.wv['montag']  
vector_montag_ac_al = w2v_ac_al.wv['montag'] 

cosine_montag = 1 - spatial.distance.cosine(vector_montag_bc, vector_montag_ac_al)
cosine_montag

0.9994910359382629

Weiteres Vorgehen 
- von allen Wörtern/Vektoren jeweils Cosinus-Ähnlichkeit berechnen
- durchschnittliche Ähnlichkeit und Standardfehler
- ist die Ähnlichkeit von "Kontaktverbot" vorher und nachher innerhalb des Standardfehlers?